# Artificial Neural Networks and Deep Learning 2022 - Homework 2
### Politecnico di Milano
Team name: bisogna_tenersi_idratati_nelle_ore_più_calde

Team members:
- Alex Amati
- Stefano Civelli
- Luca Molteni

In this notebook we will train 3 models and then we will perform voting between them at predict time

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random

tfk = tf.keras
tfkl = tf.keras.layers

Here we define all the parameters we need to train our models.

In [ ]:
val_size = 0.2
seed = 0x911
dir_models = 'models'

params = {
    'luca': {
        'window' : 34,
        'stride' : 1,
        'val_stride': 3,
        'features': ['std', 'norm', 'singlenorm', 'logstd'],
        'epochs' : 200,
        'ES_patience' : 50,
        'LR_patience' : 15,
        'LR_factor' : 0.5,
        'LR_min' : 1e-10,
        'LR' : 0.0001,
        'batch_size' : 64,
        'class_weight' : None
    },
    'ste': {
        'window' : 36,
        'stride' : 1,
        'val_stride': 1,
        'features' : ['std', 'norm', 'singlenorm', 'logstd'],
        'epochs' : 200,
        'ES_patience' : 50,
        'LR_patience' : 15,
        'LR_factor' : 0.5,
        'LR_min' : 1e-10,
        'LR' : 0.0001,
        'batch_size' : 64,
        'class_weight' : None
    },
    'alex': {
        'window': 36,
        'stride': 1,
        'val_stride': 1,
        'features': ['nothing'],
        'epochs' : 200,
        'ES_patience' : 50,
        'LR_patience' : 15,
        'LR_factor' : 0.5,
        'LR_min' : 1e-10,
        'LR' : 0.001,
        'batch_size' : 64,
        'class_weight' : None
    }
}

## Preprocessing
In this section we will load the dataset and we will perform all the neccessary trasformations to the input

In [ ]:
x = np.load('../x_train.npy')
y = np.load('../y_train.npy')

Convert target in one hot encoding

In [ ]:
oneHot = np.zeros((y.shape[0], y.max() + 1))
oneHot[np.arange(y.shape[0]), y] = 1
y = oneHot

Split the input in train set and validation set

In [ ]:
from sklearn.model_selection import train_test_split
x_train_tmp, x_val_tmp, y_train_tmp, y_val_tmp = train_test_split(x, y, test_size=val_size, random_state=seed, stratify=y)

Here we use our simple library:
- to perform and concatenate as features the transformations specified in params[model_name]['features']
- to reduce the input window performing oversampling according to params[model_name]['window', 'stride', 'val_stride']

In [ ]:
from libreriabella import preprocess_data
x_train = {}; x_val = {}; y_train = {}; y_val = {}

preproc = preprocess_data()

x_train['luca'], x_val['luca'], y_train['luca'], y_val['luca'] = preproc.preprocess_train(x_train_tmp, x_val_tmp, y_train_tmp, y_val_tmp, params['luca']['window'], params['luca']['stride'], params['luca']['val_stride'], params['luca']['features'])
x_train['ste'], x_val['ste'], y_train['ste'], y_val['ste'] = preproc.preprocess_train(x_train_tmp, x_val_tmp, y_train_tmp, y_val_tmp, params['ste']['window'], params['ste']['stride'], params['ste']['val_stride'], params['ste']['features'])
x_train['alex'], x_val['alex'], y_train['alex'], y_val['alex'] = preproc.preprocess_train(x_train_tmp, x_val_tmp, y_train_tmp, y_val_tmp, params['alex']['window'], params['alex']['stride'], params['alex']['val_stride'], params['alex']['features'])

predictParams = preproc.getPredictParams()

In [ ]:
input_shape = {}

input_shape['luca'] = x_train['luca'].shape[1:]
input_shape['ste'] = x_train['ste'].shape[1:]
input_shape['alex'] = x_train['alex'].shape[1:]

classes = y_train['luca'].shape[-1] # number of classes is always the same

## Models
In this section we will provide the code to build and train our three models

In [ ]:
def build_luca(input_shape, classes, seed, lr=0.001):
    input_layer = tfkl.Input(shape=input_shape, name='luca_input')
    layer = tfkl.GaussianDropout(0.4, seed=seed, name='luca_gdropout')(input_layer)
    layer = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='luca_conv0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer)
    layer = tfkl.MaxPooling1D(2, name='luca_pool0')(layer)
    layer = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='luca_conv1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer)
    gap1 = tfkl.GlobalAveragePooling1D(name='luca_gap1')(layer)
    layer = tfkl.MaxPooling1D(2, name='luca_pool1')(layer)
    layer = tfkl.Conv1D(16, 3, padding='same', activation='relu', name='luca_conv2', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer)
    gap0 = tfkl.GlobalAveragePooling1D(name='luca_gap0')(layer)

    concat = tfkl.Concatenate(name='luca_concat')([gap0, gap1])
    dropout =tfkl.Dropout(0.3, seed=seed, name='luca_dropout')(concat)
    classifier = tfkl.Dense(128, activation='relu', name='luca_dense', kernel_initializer=tfk.initializers.GlorotUniform(seed))(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax', name='luca_output', kernel_initializer=tfk.initializers.GlorotUniform(seed))(classifier)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='luca_model')
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(lr), metrics='accuracy')
    
    return model

In [ ]:
def build_ste(input_shape, classes, seed, lr=0.001):
    input_layer = tfkl.Input(shape=input_shape, name='luca_input')
    gdropout = tfkl.GaussianDropout(0.4, seed=seed, name='luca_gdropout')(input_layer)

    layer0 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv0-0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(gdropout)
    layer0 = tfkl.MaxPooling1D(2, name='ste_pool0')(layer0)
    layer0 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv0-1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer0)
    layer0 = tfkl.GlobalAveragePooling1D(name='ste_gap0')(layer0)
    layer0 = tfkl.Dropout(0.5, seed=seed, name='ste_dropout0')(layer0)

    layer1 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv1-0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(gdropout)
    layer1 = tfkl.MaxPooling1D(2, name='ste_pool1')(layer1)
    layer1 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv1-1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer1)
    layer1 = tfkl.GlobalAveragePooling1D(name='ste_gap1')(layer1)
    layer1 = tfkl.Dropout(0.5, seed=seed, name='ste_dropout1')(layer1)

    layer2 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv2-0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(gdropout)
    layer2 = tfkl.MaxPooling1D(2, name='ste_pool2')(layer2)
    layer2 = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='ste_conv2-1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(layer2)
    layer2 = tfkl.GlobalAveragePooling1D(name='ste_gap2')(layer2)
    layer2 = tfkl.Dropout(0.5, seed=seed, name='ste_dropout2')(layer2)

    concat = tfkl.Concatenate(name='ste_concat')([layer0, layer1, layer2])
    classifier = tfkl.Dense(128, activation='relu', name='ste_dense', kernel_initializer=tfk.initializers.GlorotUniform(seed))(concat)
    output_layer = tfkl.Dense(classes, activation='softmax', name='ste_output', kernel_initializer=tfk.initializers.GlorotUniform(seed))(classifier)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='ste_model')
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(lr), metrics='accuracy')
    
    return model

In [ ]:
def build_alex(input_shape, classes, seed, lr=0.001):
    input_layer = tfkl.Input(shape=input_shape, name='alex_input')

    cnn = tfkl.Conv1D(512, 3, padding='same', activation='relu', name='alex_b0conv0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(input_layer)
    cnn = tfkl.MaxPooling1D(2, name='alex_pool0')(cnn)
    cnn = tfkl.Conv1D(256, 3, padding='same', activation='relu', name='alex_b1conv0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    cnn = tfkl.Conv1D(256, 3, padding='same', activation='relu', name='alex_b1conv1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    cnn = tfkl.Conv1D(256, 3, padding='same', activation='relu', name='alex_b1conv2', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    cnn = tfkl.MaxPooling1D(2, name='alex_pool1')(cnn)
    cnn = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='alex_b2conv0', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    cnn = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='alex_b2conv1', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    cnn = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='alex_b2conv2', kernel_initializer=tfk.initializers.GlorotUniform(seed))(cnn)
    gap = tfkl.GlobalAveragePooling1D(name='alex_gap')(cnn)

    dropout =tfkl.Dropout(0.3, seed=seed, name='alex_dropout')(gap)
    classifier = tfkl.Dense(128, activation='relu', name='alex_dense', kernel_initializer=tfk.initializers.GlorotUniform(seed))(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax', name='alex_output', kernel_initializer=tfk.initializers.GlorotUniform(seed))(classifier)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='alex_model')
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(lr), metrics='accuracy')
    
    return model

In [ ]:
models = {}
models['luca'] = build_luca(input_shape['luca'], classes, seed, params['luca']['LR'])
models['ste'] = build_ste(input_shape['ste'], classes, seed, params['ste']['LR'])
models['alex'] = build_alex(input_shape['alex'], classes, seed, params['alex']['LR'])

In [ ]:
models['luca'].summary()
# tfk.utils.plot_model(models['luca'], expand_nested=True,  show_shapes=True, show_dtype=False, show_layer_names=True, show_layer_activations=True, to_file=models['luca'].name+'.png')

In [ ]:
models['ste'].summary()
# tfk.utils.plot_model(models['ste'], expand_nested=True,  show_shapes=True, show_dtype=False, show_layer_names=True, show_layer_activations=True, to_file=models['ste'].name+'.png')

In [ ]:
models['alex'].summary()
# tfk.utils.plot_model(models['alex'], expand_nested=True,  show_shapes=True, show_dtype=False, show_layer_names=True, show_layer_activations=True, to_file=models['alex'].name+'.png')

In [ ]:
history = {}
for key, model in models.items():
    history[key] = model.fit(
        x = x_train[key],
        y = y_train[key],
        validation_data = (x_val[key], y_val[key]),
        shuffle = True,
        epochs = params[key]['epochs'],
        batch_size = params[key]['batch_size'],
        class_weight = params[key]['class_weight'],
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=params[key]['ES_patience'], restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=params[key]['LR_patience'], factor=params[key]["LR_factor"], min_lr=params[key]['LR_min'])
        ]
    ).history

In [ ]:
# for key, model in models.items():
#     model.save(os.path.join(dir_models, key))

#     hist_df = pd.DataFrame(history[key])
#     hist_csv_file = os.path.join(dir_models, key, 'history.csv')
#     with open(hist_csv_file, mode='w') as f:
#         hist_df.to_csv(f)

## Sulauzytas - majority voting
Here we define our class to perform the majority voting between the three previous models.
Note that the model.py we uploaded on CodaLab was different because for time purpose we used a less general approach

In [ ]:
from libreriabella import preprocess_data

class Sulauzytas:
    def __init__(self, model1, features1, model2, features2, model3, features3, params):
        self._preproc = preprocess_data()
        self._preproc.setPredictParams(*params)

        self._models = [model1, model2,model3]
        self._features = [features1, features2, features3]

    def _voteBetween3WindowsSamples(self, _out):
        out = []
        for a, b, c in zip(_out[0::3], _out[1::3], _out[2::3]):
            if a == b and b == c:
                out.append(a)
            elif a == b:
                out.append(a)
            elif b == c:
                out.append(b)
            elif a == c:
                out.append(a)
            else:
                out.append(a)
        return np.array(out)

    def predict(self, x):
        for i, model in enumerate(self._models):
            assert (x.shape[1] == model.input_shape[1]) or (x.shape[1] - model.input_shape[1] == 2), "model"+str(i+1)+": (input_window - model_window) not in [0, 2] --- NOT IMPLEMENTED"
        
        inputs = []
        for i, model in enumerate(self._models):
            inputs.append(self._preproc.preprocess_predict(x, model.input_shape[1], 1, self._features[i]))

        outputs = []
        for i, (model, input) in enumerate(zip(self._models, inputs)):
            _out = model.predict(input)
            _out = tf.argmax(_out, axis=-1)
            if (model.input_shape[1] == x.shape[1] - 2):
                _out = self._voteBetween3WindowsSamples(_out)
            outputs.append(_out)
        
        a = np.array(outputs[0])
        b = np.array(outputs[1])
        c = np.array(outputs[2])

        prediction = []
        # We found using the confsion matrix that class 7 was almost always correctly predicted by the second model (b)
        for i in range(len(a)):
            if(b[i] == 7):
                prediction.append(b[i])
            elif (a[i] == b[i] or a[i] == c[i]):
                prediction.append(a[i])
            elif (b[i] == c[i]):
                prediction.append(c[i])
            elif (a[i] != b[i] != c[i]):
                prediction.append(a[i])

        numpy_pred = np.array(prediction)
        tensor_pred = tf.convert_to_tensor(numpy_pred)

        return tensor_pred

In [ ]:
sulauzytas = Sulauzytas(models['ste'], params['ste']['features'], models['luca'], params['luca']['features'], models['alex'], params['alex']['features'], predictParams)

We did't create a test set so we will try the predict function on our validation set

In [ ]:
p = sulauzytas.predict(x_val_tmp)
print(p)